In [9]:
import pandas as pd
from konlpy.tag import Okt
import re
from multiprocessing import Pool
from tqdm import tqdm
tqdm.pandas()

In [27]:
df = pd.read_csv('../crawling_data/reviews_2018.csv')
df.head(20)

,reviews,titles,years
0,"니콜라이 퓰시 감독 / 130분 / 15세 관람가\n크리스 헴스워스, 마이클 섀넌,...",12 솔져스 (12 Strong),2018
1,\n \n\n안녕하세요. 케이론입니다!\n\n영화 <12 솔져스>에 대한 후기입니...,12 솔져스 (12 Strong),2018
2,영화 속 중심인물들이 소속된 부대에 대한 설명은 아래 글을 통해 확인하실 수 있습니...,12 솔져스 (12 Strong),2018
3,토르는 헬라에게 망치를 파괴당하고 눈까지 잃는다\n그리고 아스가르드 백성들을 이끌고...,12 솔져스 (12 Strong),2018
4,"12 스트롱\n감독\n니콜라이 퍼글식\n출연\n크리스 헴스워스, 마이클 섀넌, 마이...",12 솔져스 (12 Strong),2018
5,2018년 1월 11탄\n12 솔져스\n 12 솔져스\n감독\n니콜라이 퓰...,12 솔져스 (12 Strong),2018
6,"12 솔져스\n감독\n니콜라이 퓰시\n출연\n크리스 헴스워스, 마이클 섀넌, 마이클...",12 솔져스 (12 Strong),2018
7,염력을 볼까 12 솔져스를 볼까 둘다 볼까? 고민을 하다가 염력의 평이 좋지 않기에...,12 솔져스 (12 Strong),2018
8,"12 솔져스\n감독\n니콜라이 퓰시\n출연\n크리스 헴스워스, 마이클 섀넌, 마이클...",12 솔져스 (12 Strong),2018
9,"12 솔져스\n감독 니콜라이 퓰시 출연 크리스 헴스워스, 마이클 섀넌, 마이클 페나...",12 솔져스 (12 Strong),2018


In [28]:
df.shape

(22976, 3)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22976 entries, 0 to 22975
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  22975 non-null  object
 1   titles   22976 non-null  object
 2   years    22976 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 538.6+ KB


In [31]:
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22975 entries, 0 to 22975
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  22975 non-null  object
 1   titles   22975 non-null  object
 2   years    22975 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 718.0+ KB


In [32]:
stopwords = pd.read_csv('../datasets/stopwords.csv', index_col=0)
print(stopwords)

    stopword
0          아
1          휴
2        아이구
3        아이쿠
4        아이고
..       ...
783      없다는
784      않을까
785       있지
786      있으며
787      그대로

[787 rows x 1 columns]


In [44]:
# 또 다른 불용어 추가
movie_stopwords = ['영화', '배우', '감독']
# stopwords_list = list(stopwords.stopword) + movie_stopwords
stopwords_set = set(list(stopwords.stopword) + movie_stopwords)
# 파이썬에서는 리스트보다 세트로 찾는 게 훨씬 빠르다는데 차이 못느낌


In [45]:
okt = Okt()
cleaned_sentences=[]
for sentence in tqdm(df.reviews):
    sentence = re.sub('[^가-힣 ]', '', sentence)
    token = okt.pos(sentence, stem=True)  # 단어의 원형 반환   
    df_token = pd.DataFrame(token, columns=['word', 'class']) 
    df_cleaned_token = df_token[(df_token['class'] == 'Noun') | (df_token['class'] == 'Verb') | (df_token['class'] == 'Adjective')]
    words = [word for word in df_cleaned_token['word'] if word not in stopwords_set and len(word) > 1]
    cleaned_sentence = ' '.join(words)
    cleaned_sentences.append(cleaned_sentence)
    
df['cleaned_sentences'] = cleaned_sentences
df



100%|██████████| 22975/22975 [23:39<00:00, 16.18it/s]


,reviews,titles,years,cleaned_sentences
0,"니콜라이 퓰시 감독 / 130분 / 15세 관람가\n크리스 헴스워스, 마이클 섀넌,...",12 솔져스 (12 Strong),2018,니콜라이 퓰시 관람 크리스 헴스워스 마이클 섀넌 마이클 윌리암 피츠너 전문가 평점 ...
1,\n \n\n안녕하세요. 케이론입니다!\n\n영화 <12 솔져스>에 대한 후기입니...,12 솔져스 (12 Strong),2018,안녕하다 케이론 솔져스 후기 줄거리 결말 개인 긴박하다 긴급하다 상황 이어지다 전개...
2,영화 속 중심인물들이 소속된 부대에 대한 설명은 아래 글을 통해 확인하실 수 있습니...,12 솔져스 (12 Strong),2018,중심 인물 소속 부대 설명 통해 확인 어리다 테러 뉴스 속보 기억 생생하다 늦다 손...
3,토르는 헬라에게 망치를 파괴당하고 눈까지 잃는다\n그리고 아스가르드 백성들을 이끌고...,12 솔져스 (12 Strong),2018,토르 헬라 망치 파괴 잃다 아스가르드 백성 이끌다 지구 다지 구로 오다 개눈 깔다 ...
4,"12 스트롱\n감독\n니콜라이 퍼글식\n출연\n크리스 헴스워스, 마이클 섀넌, 마이...",12 솔져스 (12 Strong),2018,스트 니콜라이 퍼글 출연 크리스 헴스워스 마이클 섀넌 마이클 트래반트 개봉 미국 리...
...,...,...,...,...
22971,"김주혁 배우의 유작이라는 것,\n흥부를 새롭게 해석했다는 것,\n차쌤의 위인 강하늘...",흥부: 글로 세상을 바꾼 자 (Heung-boo: The Revolutionist),2018,김주혁 유작 것흥 부르다 새롭다 해석 것차쌤 위인 강하늘 카메오 연하다 불다 충분하...
22972,\n<흥부> 리뷰 \n이야기는 이야기되어야한다.\n\n\n설날 연휴를 겨냥해 개봉...,흥부: 글로 세상을 바꾼 자 (Heung-boo: The Revolutionist),2018,흥부 리뷰 되어다 설날 연휴 겨냥 개봉 작품 김주혁 출연 주목 받다 작품 부르다 흥...
22973,"흥부\n감독\n조근현\n출연\n정우, 김주혁, 정진영\n개봉\n2017 대한민국\n...",흥부: 글로 세상을 바꾼 자 (Heung-boo: The Revolutionist),2018,흥부 출연 정우 김주혁 정진영 개봉 대한민국 리뷰 흥부전 나오다 배경 정치 마디 해...
22974,히에로니무스 보쉬의 기이한 세계\n감독\n데이비드 비커스태프\n개봉\n2016 영국...,히에로니무스 보쉬의 기이한 세계 (The Curious World of Hieron...,2018,히에로니무스 보쉬 기이하다 세계 데이비드 비커 스태프 개봉 영국 평점 리뷰 화가 좋...


In [46]:
df = df[['titles', 'cleaned_sentences']]
print(df.info())
df.to_csv('../processing_data/cleaned_review_2018.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22975 entries, 0 to 22975
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   titles             22975 non-null  object
 1   cleaned_sentences  22975 non-null  object
dtypes: object(2)
memory usage: 538.5+ KB
None


In [ ]:
    # # 진행 상황 확인
    # count +- 1
    # if count % 10 == 0:
    #     print('.', end='')
    # if count % 100 == 0:
    #     print('')
    
    # words = []
    # for word in df_cleaned_token['word']:
    #     if len(word) > 1:
    #         if word not in stopwords_list:
    #             words.append(word)